In [92]:
import xml.etree.ElementTree as ET
import csv
import maude, os
import json
import matplotlib.pyplot as plt

#generate maude pattern for a disco XML export file
def generatePattern(path):

    tree = ET.parse(path)
    root = tree.getroot()

    #for child in root:
    #    print(child.tag, child.attrib, child.text)

    nodes = root.find("Nodes")
    edges = root.find("Edges")

    indexes_activities = {}

    for item in nodes:
        #print(item.find("Node").attrib["activity"])
        #print(item.attrib["activity"])
        #print(item.tag)
        if item.get("activity"):
            indexes_activities[item.attrib["index"]] = item.attrib["activity"] 
        if item.tag == "StartNode":
            indexes_activities[item.attrib["index"]] = "-5"
        if item.tag == "EndNode":
            indexes_activities[item.attrib["index"]] = "-7"
        #indexes_activities['activity'] = item.attrib["activity"]

    #print(indexes_activities)


    edges_ready = []
    for item in edges:
        #print(item.attrib)
        edges_ready.append([indexes_activities[item.attrib["sourceIndex"]], indexes_activities[item.attrib["targetIndex"]]])


    #print(edges_ready)
    full_list = []
    for item in nodes:
        if item.tag == "StartNode":
            id = -5
            label = "S"
            x = item.find("Layout").get("x")
            y = item.find("Layout").get("y")
        elif item.tag == "EndNode":
            id = -7
            label = "F"
            x = item.find("Layout").get("x")
            y = item.find("Layout").get("y")
        else:

            id = item.get("activity")
            label = id
            x = item.find("Layout").get("x")
            y = item.find("Layout").get("y")
        list = {}
        #print(item.get("activity"))
        list["data"] = {}
        list["data"]["id"] = id
        list["data"]["label"] = label
        list["data"]["function"] = 'nseq'
        list["position"] = {}
        list["position"]["x"] = 5000  * float(x)
        list["position"]["y"] = 5000 * float(y)
        full_list.append(list)
    #print(edges_ready)

    #with open("text_pattern.json", "w") as json_file:
    #    json.dump(full_list, json_file, indent=4)

    pattern = ""
    list_of_edges = []
    for item in edges_ready:
        list = {}
        list["data"]={}
        list["data"]["source"] = item[0]
        list["data"]["target"] = item[1]
        list_of_edges.append(list)
    #print(list_of_edges)
    sequence = {}
    sequence["data"] = {}
    sequence["data"]["sequence"] = None
    list_of_edges.append(sequence)
    full_list_2 = full_list + list_of_edges
    #print(full_list_2)
    #with open("text_pattern.json", "w") as json_file:
    #    json.dump(full_list_2, json_file, indent=4)
   
    return(full_list_2)

Convert DISCO .xml patterns into valid importable patterns for the ASAp supervision system.

In [ ]:
import os
import json


folder_path = './Disco_process_samples'

for file_name in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file_name)
    if os.path.isfile(full_path):
        # Add the file to the list
        with open(file_name[:-4]+".json", "w") as json_file:
            json.dump(generatePattern(full_path), json_file, indent=4)   

Convert sequences from .CSV file to valid importable sequences for the ASAp supervision system

In [ ]:
import csv

full_positive = []
full_negative = []
# Open the CSV file
with open('./Sample action sequences in .CSV format/idx_sequences_val.csv', newline='') as positive_val:
    # Create a CSV reader object
    positive_val_c = csv.reader(positive_val)
    for row in positive_val_c:
        full_positive.append(row)
with open('./Sample action sequences in .CSV format/idx_sequences_test.csv', newline='') as positive_test:
    # Create a CSV reader object
    positive_test_c = csv.reader(positive_test)
    for row in positive_test_c:
        full_positive.append(row)
with open('./Sample action sequences in .CSV format/idx_negative_sequences_val.csv', newline='') as negative:
    # Create a CSV reader object
    negative_c = csv.reader(negative)
    for row in negative_c:
        full_negative.append(row)



testing_dataset = []
for i in range(400,len(full_positive)):
    testing_dataset.append(["S"] + full_positive[i] + ["F"])

testing_negative = []
for i in range(50,len(full_negative)):
    testing_negative.append(["S"] + full_negative[i] + ["F"])

for i in range(0,len(testing_dataset)):
    name = "positive_test_sequence_" + str(i+1) + ".json"
    with open(name, "w") as json_file:
        json.dump(testing_dataset[i], json_file, indent=4)

for i in range(0,len(testing_negative)):
    name = "negative_test_sequence_" + str(i+1) + ".json"
    with open(name, "w") as json_file:
        json.dump(testing_negative[i], json_file, indent=4)



   


['S', '15', '16', '17', '20', '23', '79', '80', '81', '82', '83', '129', '136', '143', '142', '141', '140', '139', '138', '137', '135', '134', '133', 'F']
